In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import os
import sys
from multiprocessing import Pool, cpu_count
from tqdm.auto import tqdm
import pandas as pd

In [63]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

In [64]:
project_root = os.path.abspath(
    os.path.join("C:/Users/jllgo/OneDrive/Documentos/UFMG/MSc & MSI 2/MatchSegmentation")
)
if project_root not in sys.path:
    sys.path.append(project_root)

In [65]:
from src.pass_networks.process_games import process_game
from src.pass_networks.process_games import load_game
from src.pass_networks.process_games import save_game
from src.pass_networks.process_match_info import get_match_info
from src.pass_networks.process_match_info import process_players
from src.pass_networks.process_intervals import get_interval_graphs
from src.pass_networks.process_intervals import load_graphs
from src.pass_networks.process_intervals import save_graphs

In [66]:
RAW_DATA_PATH = '../data/raw/PL-22-23'
INTERMEDIATE_DATA_PATH = '../data/intermediate/PL-22-23'
PROCESSED_DATA_PATH = "../data/processed/PL-22-23"
CSV_DATA_PATH = "../data/csv"

# 1. Get Data

In [67]:
games = os.listdir(RAW_DATA_PATH)
raw_game_ids = [game.split('.')[0] for game in games]
print(f"Games to process: {len(raw_game_ids)}")

Games to process: 190


In [68]:
pre_processed_games = os.listdir(INTERMEDIATE_DATA_PATH)
pre_processed_game_ids = [game.split('.')[0] for game in pre_processed_games]
print(f"Games already pre-processed: {len(pre_processed_game_ids)}")

Games already pre-processed: 112


In [69]:
processed = os.listdir(PROCESSED_DATA_PATH)
processed_game_ids = [game.split('.')[0] for game in processed]
print(f"Games already processed: {len(processed_game_ids)}")

Games already processed: 111


In [70]:
failed_ids = pd.read_csv(f"{CSV_DATA_PATH}/failed_matches.csv", header=None).values.astype(str).flatten().tolist()
failed_ids

['4437',
 '4445',
 '4448',
 '4458',
 '4446',
 '4493',
 '4499',
 '4502',
 '4509',
 '4510',
 '4524',
 '4527',
 '4528',
 '4529',
 '4530',
 '4531',
 '4532',
 '4533',
 '4534',
 '4535',
 '4536',
 '4537',
 '4538',
 '4539',
 '4540',
 '4541',
 '4542',
 '4543',
 '4544',
 '4545',
 '4546',
 '4547',
 '4548',
 '4549',
 '4550',
 '4551',
 '4552',
 '4553',
 '4554',
 '4555',
 '4556',
 '4566',
 '4567',
 '4570',
 '4572',
 '4574',
 '4578',
 '4579',
 '4580',
 '4596']

In [71]:
pre_processed_games = [game_id for game_id in pre_processed_game_ids if game_id not in failed_ids]
print(f"Pre Processed Games to Process: {len(pre_processed_games)}")

Pre Processed Games to Process: 111


In [72]:
matches_to_process = [game_id for game_id in raw_game_ids if game_id not in pre_processed_game_ids and game_id not in failed_ids][:30]
print(f"Games to process: {len(matches_to_process)}")

Games to process: 10


In [73]:
matches = []

failed_matches = []

# #Check if the game has already been pre-processed
# if len(pre_processed_game_ids) > 0:
#     tasks = [(INTERMEDIATE_DATA_PATH,game_id) for game_id in pre_processed_game_ids]

#     for task in tqdm(tasks, desc="Loading Pre-Processed Games", total=len(tasks)):
#         metadata_df, players_df, events_df = load_game(task)
#         matches.append((task[1],metadata_df, players_df, events_df))

if len(matches_to_process) > 0:

    for game_id in tqdm(matches_to_process, desc="Processing Games", total=len(matches_to_process)):
        metadata_df, players_df, events_df = process_game((RAW_DATA_PATH, game_id))
        if metadata_df.empty or players_df.empty or events_df.empty:
            failed_matches.append(game_id)
            continue
        else:
            matches.append((game_id,metadata_df, players_df, events_df))
            save_game(metadata_df,players_df,events_df,INTERMEDIATE_DATA_PATH,game_id)



Processing Games:   0%|          | 0/10 [00:00<?, ?it/s]

Loading frames from match 4597: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/175995 [00:00<?, ? frames/s]

Loading frames from match 4598: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/180924 [00:00<?, ? frames/s]

...

Validating frames:   0%|          | 0/176436 [00:00<?, ? frames/s]

Loading frames from match 4605: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/171095 [00:00<?, ? frames/s]

Loading frames from match 4606: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/173616 [00:00<?, ? frames/s]

In [74]:
failed_matches = pd.DataFrame(failed_matches, columns=["game_id"])
failed_matches.to_csv(f"{CSV_DATA_PATH}/failed_matches.csv", index=False, header=False, mode='a')

In [75]:
players_info, teams_info, games_info = get_match_info(CSV_DATA_PATH)

# 2. Data Treatment

In [ ]:
players_dfs = []
events_dfs = []
metadata_dfs = []
match_ids = []

for match_dfs in tqdm(matches, desc="Data Processing"):
    match_id, metadata_df, players_df, events_df = match_dfs
    players_match_info = players_info[players_info['match_id'] == int(match_id)].reset_index(drop=True)
    match_info = games_info[games_info['match_id'] == int(match_id)].reset_index(drop=True)

    players_df = process_players(players_df, match_info, players_match_info, metadata_df)

    players_dfs.append(players_df)
    events_dfs.append(events_df)
    metadata_dfs.append(metadata_df)
    match_ids.append(match_id)

players_df = pd.concat(players_dfs)
events_df = pd.concat(events_dfs)
metadata_df = pd.concat(metadata_dfs)

Data Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
players_interval_df = players_df.groupby(['match_id','interval_id','team_id','player_id','nickname','shirt']).agg(
    x=('x', 'mean'),
    y=('y', 'mean')
).reset_index()

players_interval_df['player_id'] = players_interval_df['player_id'].astype(int)
players_interval_df['match_id'] = players_interval_df['match_id'].astype(int)
players_interval_df['interval_id'] = players_interval_df['interval_id'].astype(int)
players_interval_df['team_id'] = players_interval_df['team_id'].astype(int)
players_interval_df['shirt'] = players_interval_df['shirt'].astype(int)

players_interval_df[players_interval_df['interval_id']==1]

,match_id,interval_id,team_id,player_id,nickname,shirt,x,y
0,4577,1,2,15,Oleksandr Zinchenko,35,-6.095556,-17.260125
1,4577,1,2,30,Gabriel Jesus,9,-25.407863,-5.989329
2,4577,1,2,157,Granit Xhaka,34,-18.722916,-9.891943
3,4577,1,2,162,Gabriel Martinelli,11,-21.028437,-15.301360
4,4577,1,2,163,Bukayo Saka,7,-20.986642,8.602386
...,...,...,...,...,...,...,...,...
16432,4595,1,20,4157,Boubacar Traoré,6,-19.078434,-8.205097
16433,4595,1,20,5082,Nathan Collins,4,-25.171813,-9.889463
16434,4595,1,20,5131,José Sá,1,-44.672403,0.189599
16435,4595,1,20,6064,Hugo Bueno,64,-23.869012,22.569298


In [ ]:
match_intervals = players_df[['match_id','interval_id','team_id','team']].drop_duplicates().reset_index(drop=True)
match_intervals = match_intervals[match_intervals['interval_id'].notna()].reset_index(drop=True)
match_intervals

,match_id,interval_id,team_id,team
0,4577,1.0,6,home
1,4577,1.0,2,away
2,4577,2.0,6,home
3,4577,2.0,2,away
4,4577,3.0,6,home
...,...,...,...,...
1579,4595,47.0,2,away
1580,4595,48.0,20,home
1581,4595,48.0,2,away
1582,4595,49.0,20,home


In [ ]:
data = [{
    'match_id':row['match_id'],
    'interval_id':row['interval_id'],
    'team_id': row['team_id'],
    'player_id': -1,
    'nickname': 'Goal',
    'shirt': -1,
    'x': 52 if row['team']=='home' else -52,
    'y': 0
} for _,row in match_intervals.iterrows()]

data = pd.DataFrame(data)

players_interval_df = pd.concat([players_interval_df, data], ignore_index=True).sort_values(['match_id','interval_id','team_id','player_id']).reset_index(drop=True)
players_interval_df

,match_id,interval_id,team_id,player_id,nickname,shirt,x,y
0,4577,1.0,2,-1,Goal,-1,-52.000000,0.000000
1,4577,1.0,2,15,Oleksandr Zinchenko,35,-6.095556,-17.260125
2,4577,1.0,2,30,Gabriel Jesus,9,-25.407863,-5.989329
3,4577,1.0,2,157,Granit Xhaka,34,-18.722916,-9.891943
4,4577,1.0,2,162,Gabriel Martinelli,11,-21.028437,-15.301360
...,...,...,...,...,...,...,...,...
19076,4595,49.0,20,5082,Nathan Collins,4,-20.298433,-7.253906
19077,4595,49.0,20,5131,José Sá,1,-45.520099,3.211650
19078,4595,49.0,20,6064,Hugo Bueno,64,-6.888931,26.067867
19079,4595,49.0,20,8079,Matheus Nunes,27,-10.217798,7.580586


In [ ]:
players_interval_df = players_interval_df[players_interval_df['interval_id'].notna()].reset_index(drop=True)
players_interval_df

,match_id,interval_id,team_id,player_id,nickname,shirt,x,y
0,4577,1.0,2,-1,Goal,-1,-52.000000,0.000000
1,4577,1.0,2,15,Oleksandr Zinchenko,35,-6.095556,-17.260125
2,4577,1.0,2,30,Gabriel Jesus,9,-25.407863,-5.989329
3,4577,1.0,2,157,Granit Xhaka,34,-18.722916,-9.891943
4,4577,1.0,2,162,Gabriel Martinelli,11,-21.028437,-15.301360
...,...,...,...,...,...,...,...,...
19076,4595,49.0,20,5082,Nathan Collins,4,-20.298433,-7.253906
19077,4595,49.0,20,5131,José Sá,1,-45.520099,3.211650
19078,4595,49.0,20,6064,Hugo Bueno,64,-6.888931,26.067867
19079,4595,49.0,20,8079,Matheus Nunes,27,-10.217798,7.580586


### Events

In [ ]:
pass_df = events_df[(events_df['possession_type']=='PASS') & (events_df['outcome']=='C')][['match_id','player_id','receiver_id','event_id','possession_id','team_id']]
pass_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
0,4577,105,1926,6227560,6093714,6
2,4577,157,1688,6227567,6093720,2
3,4577,1688,30,6227569,6093722,2
4,4577,30,162,6227572,6093725,2
5,4577,162,30,6227576,6093730,2


In [ ]:
carry_df = events_df[(events_df['possession_type']=='CARRY') & ((events_df['carry_type'].isin(['T','C'])) | ((events_df['carry_type']=='D')&(events_df['outcome'].isin(['K','B']))))][['match_id','player_id','receiver_id','event_id','possession_id','team_id']]
carry_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
69,4577,25,25,6228037,6094172,6
76,4577,15,15,6228086,6149210,2
127,4577,157,157,6228358,6149422,2
166,4577,105,105,6228552,6094697,6
167,4577,1688,1688,6228555,6094699,2


In [ ]:
shots_df = events_df[(events_df['possession_type']=='SHOT')][['match_id','player_id','receiver_id','event_id','possession_id','team_id']]
shots_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
56,4577,1916,-1,6227938,6094071,2
82,4577,1916,-1,6228117,6094257,2
159,4577,162,-1,6228508,6094655,2
186,4577,30,-1,6228645,6094796,2
190,4577,105,-1,6228690,6094838,6


In [ ]:
network_events_df = pd.concat([pass_df,carry_df, shots_df]).reset_index(drop=True)
network_events_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id
0,4577,105,1926,6227560,6093714,6
1,4577,157,1688,6227567,6093720,2
2,4577,1688,30,6227569,6093722,2
3,4577,30,162,6227572,6093725,2
4,4577,162,30,6227576,6093730,2


### Networks

In [ ]:
network_events_df['match_id'] = network_events_df['match_id'].astype(float)
network_events_df['event_id'] = network_events_df['event_id'].astype(float)
network_events_df['possession_id'] = network_events_df['possession_id'].astype(float)
metadata_df['match_id'] = metadata_df['match_id'].astype(float)
metadata_df['event_id'] = metadata_df['event_id'].astype(float)
metadata_df['possession_id'] = metadata_df['possession_id'].astype(float)

network_frame_df = network_events_df.merge(metadata_df[['match_id','interval_id','frame_id','event_id','possession_id']], on=['match_id','event_id','possession_id'], how='left')
network_frame_df.head()

,match_id,player_id,receiver_id,event_id,possession_id,team_id,interval_id,frame_id
0,4577.0,105,1926,6227560.0,6093714.0,6,1.0,9539.0
1,4577.0,157,1688,6227567.0,6093720.0,2,1.0,9738.0
2,4577.0,1688,30,6227569.0,6093722.0,2,1.0,9804.0
3,4577.0,30,162,6227572.0,6093725.0,2,1.0,9858.0
4,4577.0,162,30,6227576.0,6093730.0,2,1.0,9880.0


In [ ]:
# Group by match_id, interval_id, team_id, player_id, receiver_id to get the number of interactions between players

grouped_df = network_frame_df.groupby(['match_id','interval_id','team_id','player_id','receiver_id']).size().reset_index(name='count')
grouped_df['match_id']=grouped_df['match_id'].astype(int)
grouped_df['interval_id']=grouped_df['interval_id'].astype(int)
grouped_df['team_id']=grouped_df['team_id'].astype(int)
grouped_df['player_id']=grouped_df['player_id'].astype(int)
grouped_df['receiver_id']=grouped_df['receiver_id'].astype(int)

grouped_df.head()

,match_id,interval_id,team_id,player_id,receiver_id,count
0,4577,1,2,157,1688,1
1,4577,1,2,162,30,1
2,4577,1,2,1688,30,1
3,4577,1,2,1895,1688,1
4,4577,1,2,30,162,2


In [ ]:
grouped_df = grouped_df.merge(players_info[['match_id','player_id','nickname','shirt_number']], how='left', left_on=['match_id','player_id'], right_on=['match_id','player_id']).drop_duplicates().reset_index(drop=True)
grouped_df = grouped_df.merge(players_info[['match_id','player_id','nickname','shirt_number']], how='left', left_on=['match_id','receiver_id'], right_on=['match_id','player_id']).drop_duplicates().reset_index(drop=True)

grouped_df = grouped_df.rename(columns={'player_id_x':'player_id','nickname_x':'player_nickname','shirt_number_x':'player_shirt','nickname_y':'receiver_nickname','shirt_number_y':'receiver_shirt'})
grouped_df = grouped_df.drop(['player_id_y'], axis=1)

grouped_df['receiver_nickname'] = grouped_df['receiver_nickname'].fillna('Goal')
grouped_df['receiver_shirt'] = grouped_df['receiver_shirt'].fillna(-1)
grouped_df['receiver_shirt'] = grouped_df['receiver_shirt'].astype(int)

grouped_df.head()

,match_id,interval_id,team_id,player_id,receiver_id,count,player_nickname,player_shirt,receiver_nickname,receiver_shirt
0,4577,1,2,157,1688,1,Granit Xhaka,34,Martin Ödegaard,8
1,4577,1,2,162,30,1,Gabriel Martinelli,11,Gabriel Jesus,9
2,4577,1,2,1688,30,1,Martin Ödegaard,8,Gabriel Jesus,9
3,4577,1,2,1895,1688,1,William Saliba,12,Martin Ödegaard,8
4,4577,1,2,30,162,2,Gabriel Jesus,9,Gabriel Martinelli,11


# Graphs

In [ ]:
graph_games = os.listdir(PROCESSED_DATA_PATH)
graph_game_ids = [game.split('.')[0] for game in graph_games]
print(f"Games to process: {len(graph_game_ids)}")

Games to process: 95


In [ ]:
pre_processed_ids = [game_id for game_id in match_ids if game_id not in graph_game_ids]
print(f"Games to process graphs: {len(pre_processed_ids)}")

Games to process graphs: 16


In [ ]:
#graphs = []

# if len(graph_game_ids) > 0:

#     for game_id in tqdm(graph_game_ids, desc="Loading Graphs", total=len(graph_game_ids)):
#         match_graphs = load_graphs(game_id, PROCESSED_DATA_PATH)
#         graphs.append((task[1],match_graphs))

if len(pre_processed_ids) > 0:

    for game_id in tqdm(pre_processed_ids, desc="Processing Graphs", total=len(pre_processed_ids)):

        match_grouped_df = grouped_df[grouped_df['match_id']==int(game_id)].reset_index(drop=True)
        match_players_interval_df = players_interval_df[players_interval_df['match_id']==int(game_id)].reset_index(drop=True)

        match_graphs = get_interval_graphs(match_grouped_df, match_players_interval_df)
        save_graphs(game_id, PROCESSED_DATA_PATH, match_graphs)
        #graphs.append((game_id,match_graphs))

Processing Graphs:   0%|          | 0/16 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/49 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/51 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/49 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/48 [00:00<?, ?it/s]

...

Processing intervals:   0%|          | 0/51 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/49 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/48 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/49 [00:00<?, ?it/s]

Processing intervals:   0%|          | 0/49 [00:00<?, ?it/s]